# 02-03 - Vector Store mit Azure AI Search

In diesem Notebook werden wir uns ansehen, wie wir Azure AI Search als Vektor-Store verwenden können – einschließlich der verschiedenen Suchmethoden, die der Dienst unterstützt – und wie man dies für Retrieval Augmented Generation (RAG) mit großen Sprachmodellen einsetzen kann.

## Übersicht

1. **Erklärung: Erstellen eines AI Search-Indexes über das Azure Portal (Cosmos DB UI)**
2. **Verbindung zu Azure AI Search herstellen**
3. **Vektor-Suche verwenden**
4. **Hybride Suche**
5. **RAG mit Langchain und Azure AI Search**

## 1) Erklärung: Erstellen eines **AI Search**-Indexes über das **Azure Portal** (**Cosmos DB** UI)

Wenn bereits Daten in **Cosmos DB** vorhanden sind, kann man im **Azure Portal** direkt einen **AI Search**-Index (auch „Azure Cognitive Search Index“) anhand dieser Cosmos DB-Daten erstellen.

---

### Schritte (im Azure Portal):

1. Erstelle zunächst einen neuen **AI Search**-Service in deiner **resource group**, indem du im Azure Marketplace danach suchst

![Azure AI Search Service](images/find-ai-search.png)

Konfiguriere rasch die Ressourcengruppe, Netzwerkverbindung (public internet) und lege den Dienst an.

![Konfiguration](images/config-ai-search.png) 

2. Öffne nun die Ressource **Cosmos DB** im **Azure Portal**, in der deine Daten liegen.  
3. Im linken Menü gibt es eine Option **Integrations** und dann **Add Azure AI Search**. Klicke darauf.  
4. Wähle den zuvor erstellten **AI Search**-Service und klicke auf „connect to your data“. Halte dich an den Assistenten und überspringe „add cognitive skills“.  
5. Unter „Customize target index“ achte darauf, dass alle Spalten außer der „vector column“ den Typ „Emd.String“ haben. Für die „vector column“ ist der Datentyp **Collection(Edm.Single)** erforderlich.

**WICHTIG:** Alle Spalten müssen **Searchable** und **Retrievable** sein (für die Semantic Suche später)!

6. Klicke auf **Configure vector field** und gib z. B. 3072 als Dimension an

![AI Search Index](images/config-aisearch-index.png)

Konfiguriere Algorithmus und „vectorizer“ mit den Standardeinstellungen. Für diesen Hackathon ist keine Komprimierung nötig.  



7. Abschließend klicke auf **create indexer**. Dort kannst du einen Zeitplan festlegen, um deine Datenbasis automatisch aus Cosmos DB zu aktualisieren. Für den Hackathon reicht „once“.  
8. Sobald der Indexer erstellt ist, werden deine Cosmos DB-Dokumente indexiert. Dies kann ein paar Minuten dauern.

---

### Nach dem Erstellen des Indexers:

1. Wähle deinen **Azure Cognitive Search Service** aus.  
2. Dort siehst du:  
   - **Indexer**: Zeigt dir, ob der Indexer erfolgreich war oder fehlgeschlagen ist. Du kannst auch genauer sehen, wann er das letzte Mal lief.  
   - **Indexes**: Zeigt dir die tatsächlich erstellten Suchindizes. Ist dein Indexer erfolgreich durchgelaufen, findest du hier den neuen Index.  
   - **Spalten / Felder wählen**: Entscheide, welche Felder in der Suche auftauchen sollen (z. B. Titel, Beschreibung, Genre). Bestimme, welches Feld als Schlüssel (`key`) dient.  
   - **Optional**: Vektor-Feld definieren — falls du **Vektorfelder** (Embeddings) in deinen Cosmos-Daten hast, kannst du dieses Feld sowie dessen Dimension (z. B. 1536 für `text-embedding-ada-002`) angeben.  
   - Abschließend **Speichern** / **Erstellen**.

Sobald dies abgeschlossen ist, kannst du direkt auf diesen Index zugreifen und in diesem Notebook gegen **AI Search**-Abfragen starten.

## 2) Verbindung zu Azure AI Search herstellen

Bevor wir anfangen können, müssen wir unser Python-Umfeld entsprechend konfigurieren, damit wir auf unsere Azure-Dienste zugreifen können. Wie in den anderen Beispielen verwenden wir eine `.env`-Datei mit folgenden Schlüsseln. Die Werte liegen bei AI Search Service unter dem Tab **Settings -> Keys** oder im **Overview** Tab.

```
AZURE_AI_SEARCH_SERVICE_NAME = "<DEIN AI SEARCH NAME>"
AZURE_AI_SEARCH_ENDPOINT = "<DEIN AI SEARCH ENDPOINT URL>"
AZURE_AI_SEARCH_INDEX_NAME = "<DEIN AI SEARCH INDEX NAME>"
AZURE_AI_SEARCH_API_KEY = "<DEINE AI SEARCH ADMIN API KEY>"
```

In [ ]:
# Schritt 1: Laden der Umgebungsvariablen
import os
from dotenv import load_dotenv

if load_dotenv():
    print("Env-Datei wurde erfolgreich geladen.")
else:
    print("Keine Env-Datei gefunden.")

azure_ai_search_service_name = os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")
azure_ai_search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
azure_ai_search_index_name = os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
azure_ai_search_api_key = os.getenv("AZURE_AI_SEARCH_API_KEY")

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_completion_deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
azure_openai_embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")

print("Verwende Azure AI Search:", azure_ai_search_service_name)
print("Verwende Index:", azure_ai_search_index_name)
print("Verwende Azure OpenAI Endpoint:", azure_openai_endpoint)


## 3) Schüsselwortsuche (Keyword Search)
Für alle Suchtypen müssen wir immer zuerst ein **Search Client** initialisieren.

In [ ]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizedQuery

# SearchClient initialisieren
search_client = SearchClient(
    endpoint=azure_ai_search_endpoint,
    index_name=azure_ai_search_index_name,
    credential=AzureKeyCredential(azure_ai_search_api_key)
)

print("Azure Search Client erfolgreich initialisiert.")

#### Zuerst führen wir eine einfache Schlüsselwortsuche durch. 

Wir erhalten nur ein Ergebnis, aber das ist nicht vollständig. Es könnte sein, dass im Index irgendwo das Wort "hero" enthalten ist, beispielsweise in der Beschreibung – etwa bei "heroische Taten" oder ähnlichem.

In [ ]:
# Einfache Schlüsselwortsuche durchführen
query = "hero"

results = list(search_client.search(
    search_text=query,
    query_type="simple",
    include_total_count=True,
    top=5
))

for result in results:
    print("Film: {}".format(result["original_title"]))
    print("Genre: {}".format(result["genre"]))
    print("----------")


#### Nächster Schritt: Eine Frage stellen statt nur ein Schlüsselwort zu suchen
Nun probieren wir dasselbe erneut, aber diesmal formulieren wir eine Frage, anstatt nur nach einem einzelnen Wort zu suchen.

In [ ]:
query = "Was sind die besten Filme über Superhelden?"

results = list(search_client.search(
    search_text=query,
    query_type="simple",
    include_total_count=True,
    top=5
))

for result in results:
    print("Film: {}".format(result["original_title"]))
    print("Genre: {}".format(result["genre"]))
    print("----------")



Wie zuvor sind die Ergebnisse gemischt. Einige Filme könnten tatsächlich etwas mit Superhelden zu tun haben, andere aber nicht. Der Grund dafür ist, dass die Suche weiterhin auf Schlüsselwörtern basiert.

#### Nächster Schritt: Eine Vektor-Suche ausprobieren
Um genauere Ergebnisse zu erhalten, testen wir als Nächstes eine Vektorsuche, um die Bedeutung der Suchanfrage besser zu erfassen.

## 4) Vektor-Suche verwenden

Angenommen, die Daten liegen bereits im Cosmos DB-Container (siehe letzte Übung) und wurden über die oben beschriebenen Schritte in unseren Azure AI Search Index eingepflegt. Wir benötigen also **keinen** erneuten CSV-Import oder ähnliches.

Jetzt können wir eine Vektor-Suche gegen diesen Index durchführen. Dazu verwenden wir unsere Embedding-Funktion über Azure OpenAI


### Azure OpenAI Embeddings

Wir verwenden Azure OpenAI Embeddings, um unseren Such-String als Vektor zu kodieren. Anschliessend übergeben wir diesen Vektor an den Azure Search Dienst (Vektor-Suche).

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings

azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment=azure_openai_embedding_deployment_name
)
print("Embedding-Client mit Azure OpenAI initialisiert.")

### Einfache Vektor-Suche

Wir starten mit einer reinen Vektor-Suche, d.h. wir übergeben keinen "Search Text" an den Index, sondern nur den aus Embeddings generierten Vektor. Wir sehen uns an, welche Ergebnisse zurückkommen.

In [ ]:
user_query = "What are the best movies about superheroes?"

# Embedding erstellen
query_vector = azure_openai_embeddings.embed_query(user_query)
vector_query = VectorizedQuery(
    vector=query_vector,
    k_nearest_neighbors=5,  # Anzahl der ähnlichen Treffer
    fields="vector"  # Das Feld im Index, das Embeddings enthält
)

# Alle durchsuchbaren Felder (entsprechend den Feldern in deinem Index)
searchable_fields = [
    "original_language", "original_title", "popularity", "release_date", 
    "vote_average", "vote_count", "genre", "overview", "revenue", 
    "runtime", "tagline"
]

# Suchanfrage an Azure AI Search
search_results = search_client.search(
    search_text=None,  # Reine Vektorsuche, kein Text
    search_fields=searchable_fields,  # Suche in allen relevanten Feldern
    vector_queries=[vector_query],  # Vektorsuche aktiv
    top=5
)

results_list = list(search_results)
for result in results_list:
    print(f"Titel: {result['original_title']} | Genre: {result.get('genre', 'keine Angabe')} | Score: {result['@search.score']}")


Die Ergebnisse könnten nicht genau das sein, was wir erwarten. Die Vektor-Suche liefert hier z.B. Ähnlichkeit auf Basis von Vektoren (Embeddings). Manchmal sind die Ergebnisse gut, manchmal aber nicht 100% zutreffend, weil wir rein die Vektor-Ähnlichkeit verwenden.

## 4) Hybride Suche

Azure AI Search bietet die Möglichkeit, eine **Hybrid-Suche** durchzuführen, bei der sowohl Keywords (volltextbasiert) als auch Vektoren (Embeddings) berücksichtigt werden. Weiterhin kann der semantische Ranker von Azure Cognitive Search die Ergebnisse neu bewerten.

Dies kann bessere Ergebnisse liefern, da man einerseits das Kontextverständnis der Vektoren nutzt und andererseits die "exakten" Keyword-Treffer beibehält.

#### Füge ein Semantic Search Config hinzu:

In diesem Abschnitt richten wir **semantische Suche** ein, um die Suchergebnisse nicht nur basierend auf Schlüsselwörtern, sondern auch auf die tatsächliche Bedeutung der Anfrage zu verbessern.

Ziel:
 - Wir definieren **wichtige Felder** für die semantische Suche.
 - Dadurch können wir relevantere Treffer erzielen.

Die semantische Suche analysiert den Kontext von Suchanfragen und den Inhalt der indexierten Daten, um intelligentere Suchergebnisse zu liefern.

Gehe zum deinem Index im AI Search auf Azure Portal, und mache folgende Konfiguration:

![Semantic Search Configuration](images/semantic-config.png)

#### Durchführung der hybriden Suche

In diesem Schritt kombinieren wir sowohl den Klartext der Suchanfrage als auch den Embedding-Vektor, der die semantische Bedeutung der Anfrage repräsentiert. Azure AI Search wird diese Informationen nutzen, um die Suchergebnisse zu verbessern, indem es Keyword-Übereinstimmungen, Vektor-Ähnlichkeiten und (optional) semantische Ranking-Algorithmen kombiniert.

Durch diese hybride Suche können wir sicherstellen, dass die Ergebnisse sowohl auf exakten Schlüsselwörtern als auch auf der semantischen Bedeutung der Anfrage basieren. Dies führt zu präziseren und relevanteren Suchergebnissen.

In [ ]:
# Wir übergeben sowohl den Text als auch den Embedding-Vektor.
# Azure AI Search wird Keyword + Vektor + (optional) semantische Ranking-Algorithmen kombinieren.

user_query = "What are the best movies about superheroes?"
query_vector = azure_openai_embeddings.embed_query(user_query)
vector_query = VectorizedQuery(
    vector=query_vector,
    k_nearest_neighbors=5, 
    fields="vector"
)

hybrid_search_results = search_client.search(
    search_text=user_query,  # jetzt übergeben wir den Klartext
    vector_queries=[vector_query],
    query_type="semantic",  # semantische Suche aktivieren
    semantic_configuration_name="movies-semantic-config",
    select=["original_title", "genre"],
    top=5
)
# Füge semantische Suche hinzu

hybrid_list = list(hybrid_search_results)

for r in hybrid_list:
    print("Movie:", r["original_title"], "| Genre:", r.get("genre", "N/A"))
    print("   Score:", r["@search.score"], " Reranker Score:", r.get("@search.reranker_score", "n/a"))
    print("---")

Meistens erhalten wir mit dieser Methode (Hybrid + Semantische Reranking) bessere Ergebnisse, da die Keyword-Übereinstimmungen UND Vektor-Ähnlichkeiten UND der semantische Ranker kombiniert werden.

## 5) Retrieval Augmented Generation (RAG) + Langchain

In diesem Abschnitt wollen wir die gefundenen Dokumente (Film-Infos) als Kontext an ein grosses Sprachmodell (z.B. GPT) geben, um eine konkrete Antwort zu generieren. Dies ist das übliche RAG-Muster:

1. Frage an das System
2. Erstelle Embeddings der Frage
3. Suche in Azure AI Search (oder einer anderen Vektor-Datenbank)
4. Nimm die Top-Treffer als "Kontext" (z.B. Film-Details) und baue daraus ein Prompt
5. Frage das LLM (z.B. Azure OpenAI) mit diesem Prompt
6. Erhalte generierte Antwort, die sich explizit auf unsere (internen) Daten stützt.

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LLM-Instantz (Chat)
azure_openai = AzureChatOpenAI(
    azure_deployment=azure_openai_completion_deployment_name
)

# Unser Prompt Template (in Deutsch, mit etwas Erklärung)
prompt_template = PromptTemplate(
    input_variables=["frage", "film_kontext"],
    template="""
    Du bist ein KI-System, das auf Basis der folgenden Filmdaten antworten soll.
    Frage: {frage}
    
    Hier sind die relevanten Filminformationen, die als Kontext dienen (bitte nur daraus Fakten entnehmen und nutzen):
    {film_kontext}
    
    Antworte bitte möglichst konkret.
    """
)

# Pipeline definieren
chain = prompt_template | azure_openai | StrOutputParser()

# Beispiel-Frage
query = "What are the best movies about superheroes? Please provide a synopsis."  

# 1) Embeddings
vect_query = azure_openai_embeddings.embed_query(query)
vquery = VectorizedQuery(
    vector=vect_query,
    k_nearest_neighbors=5,
    fields="vector"
)

# 2) Suche in Azure AI Search (Semantisch + Vector)
rag_results = search_client.search(
    search_text=query,
    vector_queries=[vquery],
    query_type="semantic",
    semantic_configuration_name="movies-semantic-config",
    select=["original_title", "genre", "overview"],
    top=5
)

# Kontext für das Prompt extrahieren
rag_list = list(rag_results)
kompletter_kontext = "\n".join([
    f"- Titel: {item['original_title']}, Genre: {item.get('genre','N/A')}, Info: {item.get('overview','Keine Beschreibung')}"
    for item in rag_list
])

# 3) Prompt ausfüllen & an LLM senden
final_answer = chain.invoke({"frage": query, "film_kontext": kompletter_kontext})

print("\n=== GENERIERTE ANTWORT ===\n")
print(final_answer)


Nun sollte das große Sprachmodell (Azure OpenAI) eine Antwort generieren, die sich auf unsere gefundene Filmliste bezieht (die wir aus dem Azure AI Search Index bekommen haben). Das ist das typische RAG-Prinzip.

### Zusammenfassung
1. Wir haben **bereits vorhandene Filmdaten** (z.B. in Cosmos DB) und daraus über das Azure Portal einen Azure AI Search Index erstellt.
2. Wir nutzen Embeddings über Azure OpenAI, um eine Vektor-Suche durchzuführen.
3. Im "Hybrid"-Ansatz ergänzen wir zusätzlich die Keyword-Suche + semantisches Reranking.
4. Die Resultate fügen wir in das Prompt eines LLM ein, damit das LLM gezielt die Antwort formulieren kann, ohne "Halluzinationen".

**Fertig!**